In [ ]:
!pip install transformers

In [ ]:
# We'll need these libraries to gather and shape the data.
import requests 
import pandas as pd
from itertools import compress
import re

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# CargarDatos

In [ ]:
data_path = "/content/drive/MyDrive/sis330/Final/dataset/poems.csv"
data = pd.read_csv(data_path)
data.fillna("", inplace=True)
print(data)

                                                   poems tipo_rima
0      gracias está bien.\nPorque en las tardes con s...      abab
1      Porque en las tardes con sutil desmayo.\npiado...      abab
2      Padre viejo y triste rey de las divinas cancio...      abab
3      son en mi camino focos de una luz enigmática.\...      abab
4      tus pupilas mustias vagas de pensar y abstracc...      abab
...                                                  ...       ...
10319  sobre aquella agua de esplendor desnuda.\nse v...      abab
10320  esa ave taciturna en qué medita.\nNo ha sacudi...      abba
10321  en que rasgabas el azul de enero.\ncon tu aman...      abba
10322  donde con él soñabas y dormías.\nal recio empu...      abba
10323  nido y amor por otras compañeras.\ny tú cansad...      abba

[10324 rows x 2 columns]


In [ ]:
poems = data['poems'].to_list()
tipo_rima = data['tipo_rima'].to_list()

# Model and Tokenizader


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
                                          #,
                                         # bos_token='<|startoftext|>', 
                                         # eos_token='<|endoftext|>', 
                                         # pad_token='<|pad|>')
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>'}
tokenizer.add_special_tokens(special_tokens_dict)
#okenizer.add_special_tokens({'additional_special_tokens':['<rim>','<eol>']})
max_tokens_poema = max([len(tokenizer.encode(poema)) for poema in poems])
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:847: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
validation_start_index = int(len(poems) * 0.2)
validation_start_index

2064

# class Data set

In [ ]:
class PoetryDataset(Dataset):
    
    def __init__(self, texts, tipo_rima, tokenizer, max_length):
        
        
        self.tokenizer = tokenizer
        self.texts = texts
        self.tipo_rimas = tipo_rima

        self.max_len = max_length

    def __len__(self):        
        return len(self.texts)
    
    def __getitem__(self, index):
        
        text = self.texts[index]
        tipo_rima = self.tipo_rimas[index]
       # text = f' <BOS>{tipo_rima} <rim> {text} <EOS> '
        text = f'<BOC>{tipo_rima}<EOC>{text}'
        enc_dict = tokenizer(text, truncation=True, max_length=self.max_len, padding="max_length")
        tokenized = enc_dict['input_ids']
        
        return torch.tensor(tokenized).long()

In [ ]:
poems_train = PoetryDataset(poems[:-validation_start_index], tipo_rima[:-validation_start_index], tokenizer, max_tokens_poema)
poems_valid = PoetryDataset(poems[-validation_start_index:], tipo_rima[-validation_start_index:],tokenizer, max_tokens_poema)

len(poems_train), len(poems_valid)

(8260, 2064)

In [ ]:
poems_train[0]

tensor([   28,    34, 14010,    30,   337,   337,    28,    37, 14010,    30,
          707,   727,   815,  1329,    14,   199,  2636,   292,   278,   347,
        35561,   295, 28168,   370, 22149,    14,   199,  1134,   413,   538,
        46438,   284,   671,  3514, 18245,    14,   199,    89,  6937,   276,
        21815, 24620,   295,   308, 20159,    14,   199, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257])

In [ ]:
poems_train[100]

tensor([   28,    34, 14010,    30,   337,   526,    28,    37, 14010,    30,
          694,  9310,   258,   276, 13552,   301,   306, 20899,    14,   199,
           65,  1130, 22425, 14576,  5703, 34533,    14,   199,  2443,  9175,
          258,   312, 39090,    14,   199,   257, 10120,   268,   265,   298,
        17542,    14,   199, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257])

In [ ]:
print(tokenizer.decode([ 28,    34, 14010,    30,   337,   526,    28,    37, 14010,    30,
          694,  9310,   258,   276, 13552,   301,   306, 20899,    14,   199,
           65,  1130, 22425, 14576,  5703, 34533,    14,   199,  2443,  9175,
          258,   312, 39090,    14,   199,   257, 10120,   268,   265,   298,
        17542,    14,   199, 50257]))

<BOC>abba<EOC>caricias de la hembra que se plasma.
a todos tus deseos invencibles.
ese imposible de los imposibles.
de adorar a un fantasma.
<PAD>


In [ ]:
batch_size = 32
train_loader = DataLoader(poems_train,
                          batch_size=batch_size,
                              shuffle=True)

validation_loader = DataLoader(poems_valid,
                            batch_size=batch_size,
                            shuffle=True)

# HyperParametros


In [ ]:
device = 'cuda'
num_epochs = 40
learning_rate = 0.0001
warmup_steps = 50
total_steps = len(train_loader) * num_epochs
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

# Train

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import json

In [ ]:
def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train')

    model.train()

    for toks in loader:

        toks = toks.to(device)
        
        outputs = model(toks,
                        labels=toks
                        )
        loss = outputs[0]
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:
def evaluate(model, loader, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Evaluate')

    model.eval()

    for toks in loader:

        toks = toks.to(device)
        
        outputs = model(toks,
                        labels=toks
                        )
        loss = outputs[0]
        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:

save_best_model_path = '/content/drive/MyDrive/sis330/Final/model/best_model_state_dict.pth'
save_best_optimizer_path = '/content/drive/MyDrive/sis330/Final/model/best_optimizer_state_dict.pth'
save_last_model_path = '/content/drive/MyDrive/sis330/Final/last_model_state_dict.pth'
save_last_optimizer_path = '/content/drive/MyDrive/sis330/Final/model/last_optimizer_state_dict.pth'

In [ ]:
home_dir = '/content/drive/MyDrive/sis330/Final/model_gpt2'

In [ ]:
train_losses = []
validation_losses = []

train_perplexities = []
validation_perplexities = []

best_validation_loss = 1e+6

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)
    epoch_validation_losses = evaluate(model, validation_loader)
    
    mean_train_loss = np.mean(epoch_train_losses)
    mean_validation_loss = np.mean(epoch_validation_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    validation_losses.extend(epoch_validation_losses)
    validation_perplexities.append(np.exp(mean_validation_loss))
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'
    message += f'Validation: loss - {mean_validation_loss:.4f} | perplexity - {validation_perplexities[-1]:.3f}'
    
    print(message)
    
    if mean_validation_loss < best_validation_loss:
        
        best_validation_loss = mean_validation_loss
        
        torch.save(model.state_dict(), save_best_model_path)
        torch.save(optimizer.state_dict(), save_best_optimizer_path)
        
    else:
        break
        
    torch.save(model.state_dict(), save_last_model_path)
    torch.save(optimizer.state_dict(), save_last_optimizer_path)

    with open(home_dir + f'info_{n_epoch}.json', 'w') as file_object:

        info = {
            'message': message,
            'train_losses': train_losses,
            'validation_losses': validation_losses,
            'train_perplexities': train_perplexities,
            'validation_perplexities': validation_perplexities
        }

        file_object.write(json.dumps(info, indent=2))

Evaluate: 100%|██████████| 65/65 [00:09<00:00,  6.78it/s, loss=2.1, perplexity=8.14]


Epoch: 1
Train: loss - 2.3900 | perplexity - 10.914
Validation: loss - 2.0974 | perplexity - 8.145


Evaluate: 100%|██████████| 65/65 [00:09<00:00,  6.80it/s, loss=2.06, perplexity=7.87]


Epoch: 2
Train: loss - 1.9825 | perplexity - 7.261
Validation: loss - 2.0633 | perplexity - 7.872


Evaluate: 100%|██████████| 65/65 [00:09<00:00,  6.80it/s, loss=2.08, perplexity=7.98]

Epoch: 3
Train: loss - 1.8299 | perplexity - 6.233
Validation: loss - 2.0775 | perplexity - 7.985


# save Model


In [ ]:
torch.save(model.state_dict(), save_best_model_path)

# generaciones de salida

In [ ]:
def generate(
    model,
    tokenizer,
    abba,
    prompt,
    entry_count=1,
    entry_length=100, #maximum number of words
    top_p=0.9,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            #generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
           # generated = generated.to(device)
           # print(generated)
            prompt = f'[BOC]{abba}[EOC]{prompt}'
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
            print(len(generated))
            for i in range(entry_length):
                  
                outputs = model(generated, labels=generated)
               # print(outputs)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<EOS>"):
                    entry_finished = True

                if entry_finished:
                    
                    next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                   # print('nt',next_token)
                    generated_num = generated_num + 1
                  #  print('g',generated)
                    output_list = list(generated.squeeze().numpy())
                    print(tokenizer.decode(output_list))
                    output_text = tokenizer.decode(output_list)
                    output_text = output_text.strip(f'[BOC]{abba}[EOC]')
                    output_text = output_text.strip(f'<EOS>')
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<EOS>" 
              generated_list.append(output_text)
    return generated_list

sentence3=generate(model.to('cpu'), tokenizer, "abba","la muerte llego antes de lo previsto")
versos3=sentence3[0].split('.')

  0%|          | 0/1 [00:00<?, ?it/s]

1


100%|██████████| 1/1 [00:26<00:00, 26.69s/it]


In [ ]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):

    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits
from tqdm import tnrange as trange
from tqdm import trange


def sample_seq(model,abba, context, length, device="cpu", temperature=1, top_k=0, top_p=0.0):

    prompt = f'<BOC>{abba}<EOC>{context}'
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  #  generated = context
    with torch.no_grad():  
        for _ in range(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            next_token_logits = outputs[0][0, -1, :] / temperature
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
    return generated
sentence=sample_seq(model,"abba","",100)
sentence=tokenizer.decode(sentence[0])